# Cross-Resonance Gate

*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

This tutorial introduces how to generate optimized pulses for Cross-Resonance (CR) gate using Quanlse. Unlike the iSWAP and CZ gate implementation in previous tutorials, CR gate is implemented using an all-microwave drive. The outline of this tutorial is as follows:
- Introduction
- Preparation
- Construct Hamiltonian
- Generate and optimize pulses via Quanlse Cloud Service
- Summary

## Introduction

**Fundamentals**

Unlike some of the other gates we have seen before, the Cross-Resonance (CR) gate only uses microwaves to implement the two-qubit interaction such that we could avoid noise due to magnetic flux. The physical realization of the CR gate includes two coupled qubits with fixed frequencies. This can be done by driving the control qubit at the frequency of the target qubit. This is shown in the figure below:


![cr-circuit](figures/cr-circuit.png)



We will first look at the effective Hamiltonian of the system (for details, please refer to Ref. \[1\] ). In the doubly rotating frame, the effective Hamiltonian for cross-resonance effect in terms of the drive strength $A$, detuning $\Delta$, drive phase $\phi_0$, and coupling strength $g_{01}$ is given \[1\] (for simplicity, we choose $\hbar = 1$) :


$$
\hat{H}_{\rm eff} = \frac{A}{4\Delta}g_{01}(\cos{\phi_0}\hat{\sigma}_0^z\hat{\sigma}_1^x+\sin{\phi_0}\hat{\sigma}_0^z\hat{\sigma}_1^y).
$$

When $\phi_0=0$, the cross-resonance effect allows for effective coupling of $\hat{\sigma}^z_0\otimes\hat{\sigma}_1^x$. We can thus derive the time evolution matrix from the effective Hamiltonian above:

$$
U_{\rm CR}(\theta)=e^{-i\frac{\theta}{2}\hat{\sigma}^z_0\otimes\hat{\sigma}^x_1},
$$

where $\theta = \Omega_0 gt/(2\Delta)$ ($t$ is the gate time). We can see that the cross-resonance effect enables a conditional rotation on qubit 1 (target qubit) depending on the state of qubit 0 (control qubit).   


Following the derivation above, the matrix form of the CR gate is (refer to \[2\] for more details):
$$
U_{\rm CR}(\theta) = \begin{bmatrix} 
\cos{\frac{\theta}{2}} & -i\sin{\frac{\theta}{2}} & 0 & 0 \\
-i\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}} & 0 & 0 \\ 
0 & 0 & \cos{\frac{\theta}{2}} & i\sin{\frac{\theta}{2}} \\
0 & 0 & i\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2}} 
\end{bmatrix}.
$$


In particular, the matrix representation of a CR gate with $\theta = -\frac{\pi}{2}$ is:

$$
U_{\rm CR}(-\pi/2) = \frac{\sqrt{2}}{2} 
\begin{bmatrix}
1 & i & 0 & 0 \\
i & 1 & 0 & 0 \\
0 & 0 & 1 & -i \\
0 & 0 & -i & 1
\end{bmatrix}.
$$

**Application**

Having analyzed some of the fundamentals of the CR gate, we now switch our focus to the applications of the CR gate in quantum computing - one of which is the implementation of a CNOT gate with a CR gate and two additional single-qubit gates.

![cr-gate](figures/cr-gate.png) 

In this tutorial, we will model the system consisting of two three-level qubits and apply the drive pulse to the control qubit (qubit $q_0$) at the frequency of the target qubit (qubit $q_1$). By performing a rotating wave approximation (RWA), the Hamiltonian can be expressed as (refer to \[1\] for more details):

$$
\hat{H}_{\rm sys} = (\omega_{\rm q0}-\omega_{\rm d})\hat{a}_{0}^{\dagger }\hat{a}_0 + (\omega_{\rm q1}-\omega_{\rm d})\hat{a}_1^\dagger \hat{a}_1 + \frac{\alpha_0}{2} \hat{a}^{\dagger2}_0\hat{a}^2_0 + \frac{\alpha_1}{2} \hat{a}^{\dagger2}_1\hat{a}^2_1+\frac{g}{2}(\hat{a}_0\hat{a}_1^\dagger + \hat{a}_0^\dagger\hat{a}_1) + \Omega_0^x(t)\frac{\hat{a}^\dagger_0+\hat{a}_0}{2}.
$$

Please refer to the following chart for symbols' definitions:

| Notation | Definition |
|:--------:|:----------:|
|$\omega_{\rm qi}$ | qubit $q_i$'s frequency |
|$\omega_{\rm d}$|drive frequency|
|$\hat{a}_i^{\dagger}$ | creation operator |
|$\hat{a}_i$ | annihilation operator |
|$\alpha_i$| qubit $q_i$'s anharmonicity |
| $g$ | coupling strength |
| $\Omega_0^x$(t) | pulse on the x channel |

## Preparation

After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import Hamiltonian-related module
from Quanlse.Utils import Hamiltonian as qham
from Quanlse.Utils.Operator import driveX, number, duff

# Import optimizer for the cross-resonance gate
from Quanlse.remoteOptimizer import remoteOptimizeCr

# Import tools to analyze the result
from Quanlse.Utils.Tools import project, unitaryInfidelity

# Import numpy and math
from numpy import round
from math import pi

## Construct Hamiltonian

Now, we need to construct the above Hamiltonian using Quanlse. In Quanlse, all information regarding a Hamiltonian is stored in a dictionary. We start by defining some of the basic parameters needed for constructing a Hamiltonian dictionary: the sampling period, the number of qubits in the system, and the system's energy levels to consider. To initialize our Hamiltonian dictionary, we call the function `createHam()` from the module `Hamiltonian`.

In [ ]:
# Sampling period
dt = 2.0

# Number of qubits
qubits = 2

# System energy level
level = 3

# Initialize the Hamiltonian
ham = qham.createHam(title='cr-gate', dt=dt, qubitNum=qubits, sysLevel=level)

Now we can start constructing our Hamiltonian. Before we start, we would need to define a few constants to pass in as the function's arguments:

In [ ]:
# Parameters setting  
g = 0.0038 * (2 * pi) # Coupling strength 1
wq0  = 4.914 * (2 * pi) # Transition frequency for qubit 0, GHz
wq1 = 4.714 * (2 * pi) # Transition frequency for qubit 1, GHz
wd = wq1 # Drive frequency is the frequency for qubit 1, GHz 
anharm0 = -0.33 * (2 * pi) # Anharmonicity for qubit 0, GHz 
anharm1 = -0.33 * (2 * pi) # Anharmonicity for qubit 1, GHz 
detuning0 = wq0 - wd
detuning1 = wq1 - wd

Now we need to add the following terms to the Hamiltonian dictionary we initilized earlier:
$$
\hat{H}_{\rm drift} = (\omega_{\rm q0}-\omega_{\rm d}) \hat{a}_0^\dagger \hat{a}_0 + (\omega_{\rm q1}-\omega_{\rm d}) \hat{a}_1^\dagger \hat{a}_1 + \frac{\alpha_0}{2} \hat{a}_0^{\dagger}\hat{a}_0^{\dagger}\hat{a}_0 \hat{a}_0 + \frac{\alpha_1}{2} \hat{a}_1^{\dagger}\hat{a}_1^{\dagger}\hat{a}_1 \hat{a}_1,
$$
$$
\hat{H}_{\rm coup} = \frac{g_{01}}{2}(\hat{a}_0 \hat{a}_1^\dagger+\hat{a}^\dagger_0 \hat{a}_1).
$$

In Quanlse's `Operator` module, we have provided tools that would allow the users to construct the commonly used operators quickly. The detuning term $(\omega_{\rm q}-\omega_{\rm d})\hat{a}^\dagger \hat{a}$ and the anharmonicity term $\frac{\alpha}{2}\hat{a}^\dagger\hat{a}^\dagger \hat{a} \hat{a} $ can be respectively generated using  `number(n)` and `duff(n)` from the `Operator` module: the two functions `number(n)` and `duff(n)` return the $n \times n$ matrices for number operators and duffing operators. The coupling term, which takes the form, $\frac{g}{2}(\hat{a}_i^\dagger\hat{a}_j+\hat{a}_i\hat{a}_j^\dagger$), can be directly added to the Hamiltonian using function `addCoupling()`.

In [ ]:
# Add the detuning terms
qham.addDrift(ham, name='detuning0', onQubits=0, amp=detuning0, matrices=number(level))
qham.addDrift(ham, name='detuning1', onQubits=1, amp=detuning1, matrices=number(level))

# Add the anharmonicity terms
qham.addDrift(ham, name='anharm0', onQubits=0, amp=0.5 * anharm0, matrices=duff(level))
qham.addDrift(ham, name='anharm1', onQubits=1, amp=0.5 * anharm1, matrices=duff(level))

# Add the coupling term
qham.addCoupling(ham, 'coupling', [0, 1], g=0.5 * g)

Finally, we add the control term to the system Hamiltonian. The matrix for the control term can be defined by `driveX(level)` in the `Operator` module. 
$$
\hat{H}_{\rm ctrl} = \Omega_0^x(t)\frac{\hat{a}^\dagger_0+\hat{a}_0}{2}.
$$

In [ ]:
# Add the control term
qham.addControl(ham, name='q0-ctrlx', onQubits=0, matrices=driveX(level))

With the system Hamiltonian built, we can now move on to the optimization.

## Generate and optimize pulse via Quanlse Cloud Service

The optimization process usually takes a long time to process on local devices; however, we provide a cloud service that could speed up this process significantly. To use the Quanlse Cloud Service, the users need to acquire a token from http://quantum-hub.baidu.com.

In [ ]:
# Import tools to get access to cloud service
from Quanlse import Define

# To use remoteOptimizerCr on cloud, paste your token (a string) here
Define.hubToken = ''

To find the optimized pulse for CR gate, we use the function `remoteOptimizeCr()`, which takes the Hamiltonian we had previously defined, amplitude's bound, gate time, maximum iterations, and target infidelity. By calling `remoteOptimizeCr()`, the user can submit the optimization task to the Quanlse's server. If the user wants to further mitigate the infidelity, we encourage trying an increased gate time `tg` (the duration of a CR gate is around 200 to 400 nanoseconds). Users can also try increasing the search space by setting larger `aBound` and `maxIter`.

The gate infidelity for performance assessment throughout this tutorial is defined as ${\rm infid} = 1 - \frac{1}{d}\left|{\rm Tr}[U^\dagger_{\rm goal}P(U)]\right|$, where $U_{\rm goal}$ is exactly the target unitary transformation $U_{\rm CR}(-\pi/2)$; $d$ is the dimension of $U_{\rm goal}$; and $U$ is the unitary evolution of the three-level system defined previously. Note that $P(U)$ in particular describes the evolution projected to the computational subspace.

In [ ]:
# Set amplitude bound
aBound = (1.0, 3.0)

# Run the optimization
ham, infidelity = remoteOptimizeCr(ham, aBound=aBound, tg=200, maxIter=5, targetInfidelity=0.005)

We can visualize the generated pulse using `plotWaves()`. (details regarding `plotWaves()` are covered in [single-qubit-gate.ipynb](https://quanlse.baidu.com/#/doc/tutorial-single-qubit))  

In [ ]:
# Print waves and the infidelity
qham.plotWaves(ham, ['q0-ctrlx'])
print(f'infidelity: {infidelity}')

The users can also print the the projected evolution $P(U)$ using the following lines:

In [ ]:
# Print the projected evolution
result = qham.simulate(ham)
process2d = project(result["unitary"], qubits, level, 2)
print("The projected evolution P(U):\n", round(process2d, 2))

Moreover, for those interested in acquiring the numerical data of the generated pulse for each `dt`, use function `getPulseSequences()`, which takes a Hamiltonian dictionary and channels' names as parameters.

In [ ]:
qham.getPulseSequences(ham, 'q0-ctrlx')

## Summary

From constructing the system Hamiltonian to generating an optimized pulse on Quanlse Cloud Service, we have successfully devised a pulse to implement a cross-resonace gate with high fidelity. The users are encouraged to try parameter values different from this tutorial to obtain the optimal result.

## References 

\[1\] [Rigetti, Chad, and Michel Devoret. "Fully microwave-tunable universal gates in superconducting qubits with linear couplings and fixed transition frequencies." *Physical Review B* 81.13 (2010): 134507.](https://qulab.eng.yale.edu/documents/papers/Rigetti,%20Devoret%20-%20Fully%20Microwave-Tunable%20Universal%20Gates%20in%20Superconducting%20Qubits%20with%20Linear%20Couplings%20and%20Fixed%20Transition%20Frequencies.pdf)

\[2\] Nielsen, Michael A., and Isaac L. Chuang. Quantum Computation and Quantum Information: 10th Anniversary Edition. Cambridge University Press, 2010.